Copyright (c) 2019 [Sebastian Raschka](sebastianraschka.com)

https://github.com/rasbt/python-machine-learning-book-3rd-edition

[MIT License](https://github.com/rasbt/python-machine-learning-book-3rd-edition/blob/master/LICENSE.txt)

# Python. Uczenie maszynowe - kod źródłowy

# Chapter 8 - Applying Machine Learning To Sentiment Analysis

Zwróć uwagę, że rozszerzenie zawierające nieobowiązkowy znak wodny stanowi niewielki plugin notatnika IPython / Jupyter, który zaprojektowałem w celu powielania kodu źródłowego. Wystarczy pominąć poniższe wiersze kodu:

In [1]:
%load_ext watermark
%watermark -a "Sebastian Raschka" -u -d -v -p numpy,pandas,sklearn,nltk

Sebastian Raschka 
last updated: 2019-12-05 

CPython 3.7.1
IPython 7.10.1

numpy 1.17.4
pandas 0.25.3
sklearn 0.22
nltk 3.4.5


*Korzystanie z rozszerzenia `watermark` nie jest obowiązkowe. Możesz je zainstalować za pomocą polecenia:*  

    conda install watermark -c conda-forge  

lub  

    pip install watermark   

*Więcej informacji znajdziesz pod adresem: https://github.com/rasbt/watermark.*

<br>
<br>

### Spis treści

- [Przygotowywanie zestawu danych IMDb movie review do przetwarzania tekstu](#Przygotowywanie-zestawu-danych-IMDb-movie-review-do-przetwarzania-tekstu)
  - [Uzyskiwanie zestawu danych IMDb](#Uzyskiwanie-zestawu-danych-IMDb)
  - [Przetwarzanie wstępne zestawu danych IMDb do wygodniejszego formatu](#Przetwarzanie-wstępne-zestawu-danych-IMDb-do-wygodniejszego-formatu)
- [Wprowadzenie do modelu worka słów](#Wprowadzenie-do-modelu-worka-słów)
  - [Przekształcanie słów w wektory cech](#Przekształcanie-słów-w-wektory-cech)
  - [Ocena istotności wyrazów za pomocą ważenia częstości termów — odwrotnej częstości w tekście](#Ocena-istotności-wyrazów-za-pomocą-ważenia-częstości-termów-—-odwrotnej-częstości-w-tekście)
  - [Oczyszczanie danych tekstowych](#Oczyszczanie-danych-tekstowych)
  - [Przetwarzanie tekstu na znaczniki](#Przetwarzanie-tekstu-na-znaczniki)
- [Uczenie modelu regresji logistycznej w celu klasyfikowania tekstu](#Uczenie-modelu-regresji-logistycznej-w-celu-klasyfikowania-tekstu)
- [Praca z większą ilością danych — algorytmy sieciowe i uczenie pozardzeniowe](#Praca-z-większą-ilością-danych-—-algorytmy-sieciowe-i-uczenie-pozardzeniowe)
- [Modelowanie tematyczne za pomocą alokacji ukrytej zmiennej Dirichleta](#Modelowanie-tematyczne-za-pomocą-alokacji-ukrytej-zmiennej-Dirichleta)
  - [Rozkładanie dokumentów tekstowych za pomocą analizy LDA](#Rozkładanie-dokumentów-tekstowych-za-pomocą-analizy-LDA)
  - [Analiza LDA w bibliotece scikit-learn](#Analiza-LDA-w-bibliotece-scikit-learn)
- [Podsumowanie](#Podsumowanie)

<br>
<br>

# Przygotowywanie zestawu danych IMDb movie review do przetwarzania tekstu

## Uzyskiwanie zestawu danych IMDb

Zestaw danych IMDB movie review jest dostępny do pobrania pod adresem [http://ai.stanford.edu/~amaas/data/sentiment/](http://ai.stanford.edu/~amaas/data/sentiment/).
Po pobraniu pliku należy go rozpakować.

A) Jeżeli używasz systemu Linux lub MacOS X, otwórz nowe okno terminala i za pomocą polecenia `cd` przejdź do katalogu, w którym znajduje się pobrane archiwum, a następnie wpisz komendę: 

`tar -zxf aclImdb_v1.tar.gz`

B) W przypadku systemu Windows pobierz archiwizator (np. [7Zip](http://www.7-zip.org)), dzięki któremu otworzysz skompresowany plik.

**Alternatywny kod służący do pobrania i rozpakowania zestawu danych w środowisku Python:**

In [2]:
import os
import sys
import tarfile
import time
import urllib.request


source = 'http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz'
target = 'aclImdb_v1.tar.gz'


def reporthook(count, block_size, total_size):
    global start_time
    if count == 0:
        start_time = time.time()
        return
    duration = time.time() - start_time
    progress_size = int(count * block_size)
    speed = progress_size / (1024.**2 * duration)
    percent = count * block_size * 100. / total_size

    sys.stdout.write("\r%d%% | %d MB | %.2f MB/s | %d s szacowanego czasu" %
                    (percent, progress_size / (1024.**2), speed, duration))
    sys.stdout.flush()


if not os.path.isdir('aclImdb') and not os.path.isfile('aclImdb_v1.tar.gz'):
    urllib.request.urlretrieve(source, target, reporthook)

32% | 25 MB | 6.34 MB/s | 4 sec elapsed

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



72% | 58 MB | 7.86 MB/s | 7 sec elapsed

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



100% | 80 MB | 8.24 MB/s | 9 sec elapsed

In [3]:
if not os.path.isdir('aclImdb'):

    with tarfile.open(target, 'r:gz') as tar:
        tar.extractall()

## Przetwarzanie wstępne zestawu danych IMDb do wygodniejszego formatu

In [4]:
import pyprind
import pandas as pd
import os

# zmień wartość zmiennej `basepath` na ścieżkę do katalogu, w którym
# znajduje się nierozpakowany plik z zestawem danych

basepath = 'aclImdb'

labels = {'pos': 1, 'neg': 0}
pbar = pyprind.ProgBar(50000)
df = pd.DataFrame()
for s in ('test', 'train'):
    for l in ('pos', 'neg'):
        path = os.path.join(basepath, s, l)
        for file in sorted(os.listdir(path)):
            with open(os.path.join(path, file), 
                      'r', encoding='utf-8') as infile:
                txt = infile.read()
            df = df.append([[txt, labels[l]]], 
                           ignore_index=True)
            pbar.update()
df.columns = ['Recenzja', 'Sentyment']

0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:01:51


Tasowanie zawartości obiektu DataFrame:

In [5]:
import numpy as np

np.random.seed(0)
df = df.reindex(np.random.permutation(df.index))

Opcjonalnie: Zapisywanie zgromadzonych danych w postaci pliku CSV:

In [6]:
df.to_csv('movie_data.csv', index=False, encoding='utf-8')

In [7]:
import pandas as pd

df = pd.read_csv('movie_data.csv', encoding='utf-8')
df.head(3)

,review,sentiment
0,"In 1974, the teenager Martha Moxley (Maggie Gr...",1
1,OK... so... I really like Kris Kristofferson a...,0
2,"***SPOILER*** Do not read this, if you think a...",0


In [8]:
df.shape

(50000, 2)

---

### Uwaga

Jeżeli masz problem z utworzeniem pliku `movie_data.csv`, możesz pobrać jego zarchiwizowaną postać ze strony 
https://github.com/rasbt/python-machine-learning-book-3rd-edition/tree/master/code/ch08/.

---

<br>
<br>

# Wprowadzenie do modelu worka słów

...

## Przekształcanie słów w wektory cech

Wywołując metodę fit_transform w CountVectorizer stworzyliśmy wokabularz modelu worka słów i przekształciliśmy trzy następujące zdanie w rzadkie wektory cech:

1. Słońce grzeje dziś mocno
2. Pogoda jest dziś wiosenna
3. Słońce grzeje dziś mocno i pogoda jest dziś wiosenna, a jeden i jeden daje dwa


In [9]:
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer

count = CountVectorizer()
docs = np.array([
        'Słońce grzeje dziś mocno',
        'Pogoda jest dziś wiosenna',
        'Słońce grzeje dziś mocno i pogoda jest dziś wiosenna, a jeden i jeden daje dwa'])
bag = count.fit_transform(docs)

Sprawdźmy teraz zawartość tego wokabularza, aby lepiej zrozumieć pojęcia, jakimi będziemy się posługiwać:

In [10]:
print(count.vocabulary_)

{'the': 6, 'sun': 4, 'is': 1, 'shining': 3, 'weather': 8, 'sweet': 5, 'and': 0, 'one': 2, 'two': 7}


Jak widać powyżej, wokabularz jest przechowywany w słowniku Pythona, w którym poszczególne wyrazy otrzymują indeksy liczbowe. Spójrzmy na wygenerowane przez nas wektory cech:

Pozycja każdego indeksu w ukazanych wektorach cech odpowiada liczbom całkowitym przechowywanym jako elementy słownika CountVectorizer; np. pierwsza cecha o indeksie 0 reprezentuje zliczenia wyrazu „daje”, który pojawia się wyłącznie w ostatnim zdaniu, natomiast słowo „dziś” na trzeciej pozycji (indeks 2 w wektorach cech) występuje we wszystkich trzech zdaniach. Wartości przechowywane w wektorach cech są również nazywane częstością termów: *tf (t, d)* — liczbą wystąpień termu t w dokumencie *d*.

In [11]:
print(bag.toarray())

[[0 1 0 1 1 0 1 0 0]
 [0 1 0 0 0 1 1 0 1]
 [2 3 2 1 1 1 2 1 1]]


<br>

## Ocena istotności wyrazów za pomocą ważenia częstości termów — odwrotnej częstości w tekście

In [12]:
np.set_printoptions(precision=2)

W czasie analizowania danych tekstowych często natrafiamy na wyrazy z obydwu klas pojawiające się w wielu różnych danych tekstowych. Takie często występujące słowa zazwyczaj nie zawierają żadnych przydatnych ani rozróżniających informacji. W tym ustępie poznamy użyteczną technikę zwaną ważeniem częstości termów — odwrotną częstością w tekście (tf-idf), za pomocą której zmniejszamy wagi takich mniej istotnych wyrazów w wektorach cech. Parametr tf-idf możemy zdefiniować jako iloczyn częstości termów przez odwrotną częstość w tekście:

$$\text{tf-idf}(t,d)=\text{tf (t,d)}\times \text{idf}(t,d)$$

Tutaj tf (t, d) jest wprowadzoną w poprzednim ustępie częstością termów, natomiast odwrotną częstość w tekście idf (t, d) defi-niujemy następująco:

$$\text{idf}(t,d) = \text{log}\frac{n_d}{1+\text{df}(d, t)},$$

gdzie $n_d$ oznacza całkowitą liczbę danych tekstowych, a *df(d, t)* — liczbę danych tekstowych *d* zawierających term *t*. Zwróć uwagę, że wstawienie stałej 1 do mianownika nie jest konieczne i służy do przydzielania niezerowej wartości termom znajdującym się we wszystkich próbkach uczących; logarytm gwarantuje, że niewielkie częstości danych tekstowych nie będą otrzymywały zbyt dużej wagi.

Biblioteka scikit-learn zawiera jeszcze jedną klasę transformującą, `TfidfTransformer`, przyjmującą częstości termów przechowywane w klasie `CountVectorizer` i przekształcające je w wartości tf-idfs:

In [13]:
from sklearn.feature_extraction.text import TfidfTransformer

tfidf = TfidfTransformer(use_idf=True, 
                         norm='l2', 
                         smooth_idf=True)
print(tfidf.fit_transform(count.fit_transform(docs))
      .toarray())

[[0.   0.43 0.   0.56 0.56 0.   0.43 0.   0.  ]
 [0.   0.43 0.   0.   0.   0.56 0.43 0.   0.56]
 [0.5  0.45 0.5  0.19 0.19 0.19 0.3  0.25 0.19]]


Jak wiemy z poprzedniego ustępu, wyraz „dziś” ma największą częstość termu w trzecim dokumencie, przez co stanowi najczęściej występujące słowo. Jednak po przekształceniu tego samego wektora cech za pomocą algorytmu tf-idf widzimy, że słowo to jest powiązane teraz ze względnie niską wartością tf-idf (0,36) w trzecim dokumencie, ponieważ występuje również w pozostałych dwóch dokumentach tekstowych, zatem jest mało prawdopodobne, że jest ono nośnikiem przydatnych, rozróżniających informacji.


Gdybyśmy jednak ręcznie policzyli wartości tf-idf poszczególnych termów w wektorach cech, okazałoby się, że klasa `TfidfTransformer` oblicza je nieco inaczej niż standardowe, omówione wcześniej wzory. Równanie na odwrotną częstość tekstów zaimplementowane w bibliotece scikit-learn wygląda następująco:

$$\text{idf} (t,d) = log\frac{1 + n_d}{1 + \text{df}(d, t)}$$

Z kolei wzór na model tf-idf używany w bibliotece scikit-learn wygląda następująco:

$$\text{tf-idf}(t,d) = \text{tf}(t,d) \times (\text{idf}(t,d)+1)$$

Zazwyczaj częstości termów są normalizowane jeszcze przed obliczeniem wartości tf-idf, ale klasa `TfidfTransformer` normalizuje wyniki tf-idf bezpośrednio.

Domyślnie (`norm='l2'`) przeprowadzana jest normalizacja L2, która zwraca wektor o długości 1, poprzez podzielenie nieznormalizowanego wektora cech v przez normę L2:

$$v_{\text{norm}} = \frac{v}{||v||_2} = \frac{v}{\sqrt{v_{1}^{2} + v_{2}^{2} + \dots + v_{n}^{2}}} = \frac{v}{\big (\sum_{i=1}^{n} v_{i}^{2}\big)^\frac{1}{2}}$$

Aby upewnić się, że rozumiemy mechanizm działania klasy `TfidfTransformer`, przyjrzyjmy się przykładowi i wyliczmy wartość tf-idf wyrazu „dziś” w trzecim dokumencie.

Wyraz „dziś” ma w trzecim dokumencie częstość termu równą 2 (tf = 2), jego częstość występowania w tekście również wynosi 3, ponieważ znajduje się we wszystkich trzech zdaniach (df = 3). Możemy więc wyliczyć parametr idf następująco:

$$\text{idf}("dziś", d3) = log \frac{1+3}{1+3} = 0$$

Teraz w celu obliczenia wartości tf-idf wystarczy dodać 1 do odwrotnej częstości w dokumencie i pomnożyć wynik przez częstość termu:

$$\text{tf-idf}("dziś",d3)= 2 \times (0+1) = 2$$

In [14]:
tf_is = 3
n_docs = 3
idf_is = np.log((n_docs+1) / (3+1))
tfidf_is = tf_is * (idf_is + 1)
print('wartość tf-idf termu "dziś" = %.2f' % tfidf_is)

tf-idf of term "is" = 3.00


Po przeprowadzeniu analogicznych obliczeń dla pozostałych termów w trzecim dokumencie uzyskamy następujący wektor tf-idf: [
1,69, 1,69, 2  , 1,29, 3,39, 1,29, 1,29, 1,29, 1,29, 1,29]. Widać jednak, że wartości w tym wektorze cech różnią się od wyników uzyskanych za pomocą klasy `TfidfTransformer`. Musimy jeszcze tylko przeprowadzić normalizację L2, którą wykonamy w sposób zaprezentowany poniżej:

$$\text{tfi-df}_{norm} = \frac{[1.69, 1.69, 2.0, 1.29, 3.39, 1.29, 1.29 , 1.29, 1.29, 1.29]}{\sqrt{[1.69^2, 1.69^2, 2.0^2, 1.29^2, 3.39^2, 1.29^2, 1.29^2 , 1.29^2, 1.29^2, 1.29^2]}}$$

$$=[0.3, 0.3, 0.36, 0.23, 0.61, 0.23, 0.23, 0.23, 0.23, 0.23]$$

$$\Rightarrow \text{tfi-df}_{norm}("dziś", d3) = 0.45$$

Jak widać, uzyskane wyniki są zgodne z rezultatami zwracanymi przez klasę `TfidfTransformer` (poniżej). Skoro już wiemy, jak są wyliczane wartości tf-idf, możemy przejść do dalszej części rozdziału i wykorzystać omówione koncepcje w odniesieniu do naszego zestawu recenzji filmów.

In [15]:
tfidf = TfidfTransformer(use_idf=True, norm=None, smooth_idf=True)
raw_tfidf = tfidf.fit_transform(count.fit_transform(docs)).toarray()[-1]
raw_tfidf 

array([3.39, 3.  , 3.39, 1.29, 1.29, 1.29, 2.  , 1.69, 1.29])

In [16]:
l2_tfidf = raw_tfidf / np.sqrt(np.sum(raw_tfidf**2))
l2_tfidf

array([0.5 , 0.45, 0.5 , 0.19, 0.19, 0.19, 0.3 , 0.25, 0.19])

<br>

## Oczyszczanie danych tekstowych

In [17]:
df.loc[0, 'Recenzja'][-50:]

'is seven.<br /><br />Title (Brazil): Not Available'

In [18]:
import re
def preprocessor(text):
    text = re.sub('<[^>]*>', '', text)
    emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)',
                           text)
    text = (re.sub('[\W]+', ' ', text.lower()) +
            ' '.join(emoticons).replace('-', ''))
    return text

In [19]:
preprocessor(df.loc[0, 'Recenzja'][-50:])

'is seven title brazil not available'

In [20]:
preprocessor("</a>To :) jest :( test :-)!")

'this is a test :) :( :)'

In [21]:
df['Recenzja'] = df['Recenzja'].apply(preprocessor)

<br>

## Przetwarzanie tekstu na znaczniki

In [22]:
from nltk.stem.porter import PorterStemmer

porter = PorterStemmer()

def tokenizer(text):
    return text.split()


def tokenizer_porter(text):
    return [porter.stem(word) for word in text.split()]

In [23]:
tokenizer('runners like running and thus they run')

['runners', 'like', 'running', 'and', 'thus', 'they', 'run']

In [24]:
tokenizer_porter('runners like running and thus they run')

['runner', 'like', 'run', 'and', 'thu', 'they', 'run']

In [25]:
import nltk

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/sebastian/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [26]:
from nltk.corpus import stopwords

stop = stopwords.words('english')
[w for w in tokenizer_porter('a runner likes running and runs a lot')[-10:]
if w not in stop]

['runner', 'like', 'run', 'run', 'lot']

<br>
<br>

# Uczenie modelu regresji logistycznej w celu klasyfikowania tekstu

Usuwa znaczniki HTML i znaki interpunkcyjne w celu przyspieszenia przeszukiwania siatki:

In [27]:
X_train = df.loc[:25000, 'Recenzja'].values
y_train = df.loc[:25000, 'Sentyment'].values
X_test = df.loc[25000:, 'Recenzja'].values
y_test = df.loc[25000:, 'Sentyment'].values

In [28]:
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import GridSearchCV

tfidf = TfidfVectorizer(strip_accents=None,
                        lowercase=False,
                        preprocessor=None)

param_grid = [{'vect__ngram_range': [(1, 1)],
               'vect__stop_words': [stop, None],
               'vect__tokenizer': [tokenizer, tokenizer_porter],
               'clf__penalty': ['l1', 'l2'],
               'clf__C': [1.0, 10.0, 100.0]},
              {'vect__ngram_range': [(1, 1)],
               'vect__stop_words': [stop, None],
               'vect__tokenizer': [tokenizer, tokenizer_porter],
               'vect__use_idf':[False],
               'vect__norm':[None],
               'clf__penalty': ['l1', 'l2'],
               'clf__C': [1.0, 10.0, 100.0]},
              ]

lr_tfidf = Pipeline([('vect', tfidf),
                     ('clf', LogisticRegression(random_state=0, solver='liblinear'))])

gs_lr_tfidf = GridSearchCV(lr_tfidf, param_grid,
                           scoring='accuracy',
                           cv=5,
                           verbose=2,
                           n_jobs=-1)

**Ważna uwaga na temat parametru `n_jobs`**

Bardzo zalecane jest użycie parametru `n_jobs=-1` (zamiast `n_jobs=1`) w powyższym listingu aby wykorzystać wszystkie dostępne rdzenie procesora i przyśpieszyć przeszukiwanie siatki. Jednak część użytkowników systemu Windows zgłaszała problemy z działaniem kodu przy wyznaczonym parametrze `n_jobs=-1` co ma związek z wielordzeniowym konserwowaniem funkcji `tokenizer` i `tokenizer_porter` w systemie Windows. Można ten problem obejść zastępując funkcje  `[tokenizer, tokenizer_porter]` funkcją `[str.split]`. W konsekwencji jednak nie będziemy mogli korzystać z rdzeniowania wyrazów.

**Ważna uwaga na temat czasu przetwarzania kodu**

Realizacja poniższej komórki **może zająć od 30 do 60 minut** w zależności od konfiguracji sprzętowej, ponieważ zgodnie ze zdefiniowanymi parametrami siatki istnieje 2*2*2*3*5 + 2*2*2*3*5 = 240 modeli do wytrenowania.

Jeżeli nie chcesz czekać tak długo, możesz zmniejszyć rozmiar zestawu danych ograniczając liczbę przykładów uczących, np. w następujący sposób:

    X_train = df.loc[:2500, 'Recenzja'].values
    y_train = df.loc[:2500, 'Sentyment'].values
    
Pamiętaj jednak, że tak ograniczony rozmiar zestawu uczącego będzie oznaczał kiepską skuteczność modeli. Możesz ewentualnie usunąć parametry z powyższej siatki, aby zmniejszyć liczbę modeli - np. tak, jak pokazano poniżej:

    param_grid = [{'vect__ngram_range': [(1, 1)],
                   'vect__stop_words': [stop, None],
                   'vect__tokenizer': [tokenizer],
                   'clf__penalty': ['l1', 'l2'],
                   'clf__C': [1.0, 10.0]},
                  ]

In [29]:
gs_lr_tfidf.fit(X_train, y_train)

Fitting 5 folds for each of 48 candidates, totalling 240 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
/Users/sebastian/miniconda3/lib/python3.7/site-packages/joblib/externals/loky/process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning
[Parallel(n_jobs=-1)]: Done  17 tasks      | elapsed:  4.1min
[Parallel(n_jobs=-1)]: Done 138 tasks      | elapsed: 24.2min
[Parallel(n_jobs=-1)]: Done 240 out of 240 | elapsed: 41.7min finished


GridSearchCV(cv=5, error_score=nan,
             estimator=Pipeline(memory=None,
                                steps=[('vect',
                                        TfidfVectorizer(analyzer='word',
                                                        binary=False,
                                                        decode_error='strict',
                                                        dtype=<class 'numpy.float64'>,
                                                        encoding='utf-8',
                                                        input='content',
                                                        lowercase=False,
                                                        max_df=1.0,
                                                        max_features=None,
                                                        min_df=1,
                                                        ngram_range=(1, 1),
                                                        n

In [30]:
print('Zestaw najlepszych parametrów: %s ' % gs_lr_tfidf.best_params_)
print('Dokładność sprawdzianu krzyżowego: %.3f' % gs_lr_tfidf.best_score_)

Best parameter set: {'clf__C': 10.0, 'clf__penalty': 'l2', 'vect__ngram_range': (1, 1), 'vect__stop_words': None, 'vect__tokenizer': <function tokenizer at 0x7fbc89ae1ea0>} 
CV Accuracy: 0.897


In [31]:
clf = gs_lr_tfidf.best_estimator_
print('Dokładność testu: %.3f' % clf.score(X_test, y_test))

Test Accuracy: 0.899


<hr>
<hr>

####  Początek komentarza:
    
Zwróć uwagę, że `gs_lr_tfidf.best_score_` daje uśredniony wynik k-krotnego sprawdzianu krzyżowego. Np. gdybyśmy korzystali z obiektu `GridSearchCV` składającego się z pięciokrotnej kroswalidacji (tak jak w powyższym przykładzie), atrybut `best_score_` będzie zwracał uśredniony wynik po wyznaczeniu najlepszego modelu za pomocą pięciokrotnej walidacji krzyżowej. Wyjaśnię to na przykładzie:

In [32]:
from sklearn.linear_model import LogisticRegression
import numpy as np

from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score

np.random.seed(0)
np.set_printoptions(precision=6)
y = [np.random.randint(3) for i in range(25)]
X = (y + np.random.randn(25)).reshape(-1, 1)

cv5_idx = list(StratifiedKFold(n_splits=5, shuffle=False).split(X, y))
    
lr = LogisticRegression(random_state=123, multi_class='ovr', solver='lbfgs')
cross_val_score(lr, X, y, cv=cv5_idx)

/Users/sebastian/miniconda3/lib/python3.7/site-packages/sklearn/model_selection/_split.py:296: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  FutureWarning


array([0.4, 0.2, 0.6, 0.2, 0.4])

Po uruchomieniu powyższego kodu stworzyliśmy prosty zestaw danych składający się z losowych liczb całkowitych, które będą symbolizować etykiety klas. Następnie przekazaliśmy indeksy podzbiorów pięciokrotnego sprawdzianu krzyżowego (`cv3_idx`) funkcji zliczającej `cross_val_score`, która zwróciła pięć wyników dokładności - jest to pięć wartości wyliczonych dla poszczególnych podzbiorów testowych.

Użyjmy teraz obiektu `GridSearchCV` i przekażmy mu te same pięć podzbiorów pięciokrotnej kroswalidacji (poprzez indeksy `cv3_idx`):

In [33]:
from sklearn.model_selection import GridSearchCV

lr = LogisticRegression(solver='lbfgs', multi_class='ovr', random_state=1)
gs = GridSearchCV(lr, {}, cv=cv5_idx, verbose=3).fit(X, y) 

Fitting 5 folds for each of 1 candidates, totalling 5 fits
[CV]  ................................................................
[CV] .................................... , score=0.400, total=   0.0s
[CV]  ................................................................
[CV] .................................... , score=0.200, total=   0.0s
[CV]  ................................................................
[CV] .................................... , score=0.600, total=   0.0s
[CV]  ................................................................
[CV] .................................... , score=0.200, total=   0.0s
[CV]  ................................................................
[CV] .................................... , score=0.400, total=   0.0s


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:    0.0s finished


Jak widać, wyniki dla pięciu podzbiorów są identyczne, jak wyliczone wcześniej przez funkcję `cross_val_score`.

Atrybut `best_score_` (dostępny po wytrenowaniu klasyfikatora) obiektu `GridSearchCV` zwraca uśredniony wynik dokładności dla najlepszego modelu:

In [34]:
gs.best_score_

0.36000000000000004

Wynik ten jest, jak widać, zgodny z uśrednionym wynikiem dokładności wyliczonym za pomocą funkcji `cross_val_score`.

In [35]:
lr = LogisticRegression(solver='lbfgs', multi_class='ovr', random_state=1)
cross_val_score(lr, X, y, cv=cv5_idx).mean()

0.36000000000000004

#### Koniec komentarza.

<hr>
<hr>

<br>
<br>

# Praca z większą ilością danych — algorytmy sieciowe i uczenie pozardzeniowe

In [36]:
# Komórka ta nie jest opisana w książce, lecz
# została dodana dla Twojej wygody, żebyć mógł
# ropocząć realizowanie kodu od tego miejsca, bez
# konieczności uruchamiania wcześniejszych komórek.

import os
import gzip


if not os.path.isfile('movie_data.csv'):
    if not os.path.isfile('movie_data.csv.gz'):
        print('Umieść tu kopię archiwum movie_data.csv.gz'
              'Uzyskasz do niej dostęp'
              'a) realizując kod na początku tego notatnika'
              'lub b) pobierając go z serwisu GitHub:'
              'https://github.com/rasbt/python-machine-learning-'
              'book-2nd-edition/blob/master/code/ch08/movie_data.csv.gz')
    else:
        with gzip.open('movie_data.csv.gz', 'rb') as in_f, \
                open('movie_data.csv', 'wb') as out_f:
            out_f.write(in_f.read())

In [37]:
import numpy as np
import re
from nltk.corpus import stopwords


# Funkcja `stop` została zdefiniowana wcześniej w tym rozdziale.
# Umieściliśmy ją tu dla Twojej wygody, dzięki czemu niniejszy podrozdział
# może być traktowany niezależnie od pozostałych fragmentów notatnika w
# tym katalogu
stop = stopwords.words('english')


def tokenizer(text):
    text = re.sub('<[^>]*>', '', text)
    emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)', text.lower())
    text = re.sub('[\W]+', ' ', text.lower()) +\
        ' '.join(emoticons).replace('-', '')
    tokenized = [w for w in text.split() if w not in stop]
    return tokenized


def stream_docs(path):
    with open(path, 'r', encoding='utf-8') as csv:
        next(csv)  # pomija nagłówek
        for line in csv:
            text, label = line[:-3], int(line[-2])
            yield text, label

In [38]:
next(stream_docs(path='movie_data.csv'))

('"In 1974, the teenager Martha Moxley (Maggie Grace) moves to the high-class area of Belle Haven, Greenwich, Connecticut. On the Mischief Night, eve of Halloween, she was murdered in the backyard of her house and her murder remained unsolved. Twenty-two years later, the writer Mark Fuhrman (Christopher Meloni), who is a former LA detective that has fallen in disgrace for perjury in O.J. Simpson trial and moved to Idaho, decides to investigate the case with his partner Stephen Weeks (Andrew Mitchell) with the purpose of writing a book. The locals squirm and do not welcome them, but with the support of the retired detective Steve Carroll (Robert Forster) that was in charge of the investigation in the 70\'s, they discover the criminal and a net of power and money to cover the murder.<br /><br />""Murder in Greenwich"" is a good TV movie, with the true story of a murder of a fifteen years old girl that was committed by a wealthy teenager whose mother was a Kennedy. The powerful and rich f

In [39]:
def get_minibatch(doc_stream, size):
    docs, y = [], []
    try:
        for _ in range(size):
            text, label = next(doc_stream)
            docs.append(text)
            y.append(label)
    except StopIteration:
        return None, None
    return docs, y

In [40]:
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.linear_model import SGDClassifier


vect = HashingVectorizer(decode_error='ignore', 
                         n_features=2**21,
                         preprocessor=None, 
                         tokenizer=tokenizer)

In [41]:
from distutils.version import LooseVersion as Version
from sklearn import __version__ as sklearn_version

clf = SGDClassifier(loss='log', random_state=1)


doc_stream = stream_docs(path='movie_data.csv')

In [42]:
import pyprind
pbar = pyprind.ProgBar(45)

classes = np.array([0, 1])
for _ in range(45):
    X_train, y_train = get_minibatch(doc_stream, size=1000)
    if not X_train:
        break
    X_train = vect.transform(X_train)
    clf.partial_fit(X_train, y_train, classes=classes)
    pbar.update()

0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:20


In [43]:
X_test, y_test = get_minibatch(doc_stream, size=5000)
X_test = vect.transform(X_test)
print('Dokładność: %.3f' % clf.score(X_test, y_test))

Accuracy: 0.868


In [44]:
clf = clf.partial_fit(X_test, y_test)

## Modelowanie tematyczne za pomocą alokacji ukrytej zmiennej Dirichleta

### Rozkładanie dokumentów tekstowych za pomocą analizy LDA

### Analiza LDA w bibliotece scikit-learn

In [45]:
import pandas as pd

df = pd.read_csv('movie_data.csv', encoding='utf-8')
df.head(3)

,review,sentiment
0,"In 1974, the teenager Martha Moxley (Maggie Gr...",1
1,OK... so... I really like Kris Kristofferson a...,0
2,"***SPOILER*** Do not read this, if you think a...",0


In [46]:
from sklearn.feature_extraction.text import CountVectorizer

count = CountVectorizer(stop_words='english',
                        max_df=.1,
                        max_features=5000)
X = count.fit_transform(df['Recenzja'].values)

In [47]:
from sklearn.decomposition import LatentDirichletAllocation

lda = LatentDirichletAllocation(n_components=10,
                                random_state=123,
                                learning_method='batch')
X_topics = lda.fit_transform(X)

In [48]:
lda.components_.shape

(10, 5000)

In [49]:
n_top_words = 5
feature_names = count.get_feature_names()

for topic_idx, topic in enumerate(lda.components_):
    print("Temat %d:" % (topic_idx + 1))
    print(" ".join([feature_names[i]
                    for i in topic.argsort()\
                        [:-n_top_words - 1:-1]]))

Topic 1:
worst minutes awful script stupid
Topic 2:
family mother father children girl
Topic 3:
american war dvd music tv
Topic 4:
human audience cinema art sense
Topic 5:
police guy car dead murder
Topic 6:
horror house sex girl woman
Topic 7:
role performance comedy actor performances
Topic 8:
series episode war episodes tv
Topic 9:
book version original read novel
Topic 10:
action fight guy guys cool


Na podstawie pięciu najważniejszych wyrazów dla każdego tematu możemy zgadywać, że model LDA wykrył następujące tematy:
    
1. Generalnie kiepskie filmy (w rzeczywistości nie jest to kategoria).
2. Filmy o rodzinie.
3.	Filmy wojenne.
4.	Filmy artystyczne.
5.	Kryminały.
6.	Horrory.
7.	Komedie.
8.	Seriale lub filmy powiązane z serialami.
9.	Adaptacje książek.
10.	Filmy sensacyjne.

Aby upewnić się, że kategorie tworzone na podstawie recenzji mają sens, sprawdźmy trzy filmy oznaczone jako horrory (kategoria szósta, indeks 5):

In [50]:
horror = X_topics[:, 5].argsort()[::-1]

for iter_idx, movie_idx in enumerate(horror[:3]):
    print('\nHorror numer #%d:' % (iter_idx + 1))
    print(df['Recenzja'][movie_idx][:300], '...')


Horror movie #1:
House of Dracula works from the same basic premise as House of Frankenstein from the year before; namely that Universal's three most famous monsters; Dracula, Frankenstein's Monster and The Wolf Man are appearing in the movie together. Naturally, the film is rather messy therefore, but the fact that ...

Horror movie #2:
Okay, what the hell kind of TRASH have I been watching now? "The Witches' Mountain" has got to be one of the most incoherent and insane Spanish exploitation flicks ever and yet, at the same time, it's also strangely compelling. There's absolutely nothing that makes sense here and I even doubt there  ...

Horror movie #3:
<br /><br />Horror movie time, Japanese style. Uzumaki/Spiral was a total freakfest from start to finish. A fun freakfest at that, but at times it was a tad too reliant on kitsch rather than the horror. The story is difficult to summarize succinctly: a carefree, normal teenage girl starts coming fac ...


Za pomocą powyższego kodu wyświetliliśmy pierwszych 300 znaków z recenzji trzech pierw-szych horrorów i widzimy, że recenzje te — mimo że nie znamy tytułów filmów — wyglądają na opisy horrorów (chociaż w przypadku horroru numer 2 równie dobrze moglibyśmy zaklasy-fikować go kategorii 1.: zasadniczo kiepskie filmy).

<br>
<br>

# Podsumowanie

...

---

Czytelnicy mogą zignorować poniższą komórkę.

In [51]:
! python ../.convert_notebook_to_script.py --input r08.ipynb --output r08.py

[NbConvertApp] Converting notebook ch08.ipynb to script
[NbConvertApp] Writing 24745 bytes to ch08.py
